In [1]:
from gget import ref, search, info, seq, blast, muscle
%load_ext blackcellmagic

In [6]:
seq(["ENST00000624044"], seqtype="transcript", isoforms=False)

11 Apr 2022 22:06:23 Requesting amino acid sequence of ENST00000624044 from UniProt.
No UniProt transcript sequences were found for these Ensembl ID(s).


[]